# Tiling
Tile an image into 256 x 256 chunks

In [6]:
import matplotlib.pyplot as plt
# Imports
from tqdm import tqdm
import numpy as np
import rasterio as rio
import earthpy.spatial as es

In [7]:


def read_image(file_name) -> np.ndarray:
    """
    Read image from file_name
    Args: file_name: image file name
    Returns: image array
    """
    try:
        if not Path(file_name).is_file():
            print(file_name + "Cannot open file!")
            return None
        with rio.open(file_name) as img :
            img = img.read()
            return img
    except Exception as e:
        print("Error in read_image: " + str(e))
        return None


def save_image(img_arr, file_name, crs) -> str:
    """
    Save image to file_name
    Args: img_arr: image array
    Output: file_name: image file name
    """
    if Path(file_name).is_file():
        print(f"Overwrite existing file: {file_name}")
    with rio.open(file_name,
                    'w',
                  driver='GTiff',
                  count=img_arr.shape[0],
                  height=img_arr.shape[1],
                  width=img_arr.shape[2],
                  dtype=img_arr.dtype,
                  crs=crs) as dest:
        dest.write(img_arr)
    return file_name


def save_png_image(file_name) -> str:
    """
    Save geotiff image to png
    Args: file_name: geotiff file_name
    Output: file_name: image file name
    """
    with rio.open(file_name) as infile:
        profile=infile.profile
        png_filename=file_name.with_suffix('.png')
        raster=infile.read([6,4,2])
        with rio.open(png_filename, 'w',
                  driver='PNG',
                  count=raster.shape[0],
                  height=raster.shape[1],
                  width=raster.shape[2],
                  dtype=raster.dtype,
                  crs=profile['crs']) as dst:
            dst.write(raster)
    return file_name


def count_files(folder_path):
    """
    Count the number of files in the folder
    Args: folder_path: folder path
    Returns: number of files
    """
    count = 0
    for path in Path(folder_path).iterdir():
        if path.is_file():
            count += 1
    return count


# DEPRECATED
def padding_image(img, stride) -> np.ndarray:
    """
    Padding image to the size of multiple of stride
    Args:
        img: image array
        stride: stride
    Returns:
        padded image array
    """

    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
    height = img.shape[0]
    width = img.shape[1]
    D = img.shape[2]  # this one is for (H, W, C) format
    # get the minial padding image size
    H = int(np.ceil(height / stride) * stride)
    W = int(np.ceil(width / stride) * stride)

    padded_img = np.zeros([H, W, D], dtype=img.dtype)
    for d in range(D):  # padding every layer
        onelayer = img[:, :, d]
        padded_img[:, :, d] = np.pad(
            onelayer, ((0, H - height), (0, W - width)), "reflect"
        )
    padded_img = np.squeeze(padded_img)  # Remove axes of length one
    return padded_img

In [8]:


def tile_image(image_path, save_path, crop_size = 256, repetition_rate=0, overwrite=True) -> int:
    """
    Tile large image

    # Arguments
        image_path: image path
        save_path: save path
        crop_size: crop size
        repetition_rate: repetition rate
        overwrite: overwrite existing files

    Returns:
        number of tiles
    """

    # check crs
    crs = es.crs_check(image_path)

    # check input image
    img = read_image(image_path)
    if img is None:
        return 0
    # get image suffix
    ext = Path(image_path).suffix
    print(ext)
    # check output folder, if not exists, creat it.
    Path(save_path).mkdir(parents=True, exist_ok=True)

    print(f"Input Image File Shape (D, H, W):{ img.shape}")

    stride = int(crop_size * (1 - repetition_rate))
    print(f"crop_size = {crop_size}, stride = {stride}")

    # Ignoring pixels outside "nice" shape
    H = img.shape[1] - (img.shape[1]%stride)
    W = img.shape[2] - (img.shape[2]%stride)
    print(f"Selected Image File Shape (D, H, W): {img.shape[0]}, { H }, { W }")

    if overwrite:
        new_name = 1
    else:
        cnt = count_files(save_path)
        new_name = cnt + 1
        print(f"There are {cnt} files in the {save_path}")
        print(f"New image name will start with {new_name}")

    n_rows = int((H - crop_size) / stride + 1)
    n_cols = int((W - crop_size) / stride + 1)
    print(n_rows)
    print(n_cols)

    def tile_generator():
        for idh in range(n_rows):
            h = idh * stride
            for idw in range(n_cols):
                w = idw * stride
                yield h, w

    with tqdm(
        total=n_rows * n_cols, desc="Generating", colour="green", leave=True, unit="img"
    ) as pbar:
        for n, (h, w) in enumerate(tile_generator()):
            crop_img = img[:, h : h + crop_size, w : w + crop_size]
            crop_image_name = f"{new_name:04d}{ext}"
            crop_image_path = Path(save_path) / crop_image_name
            file_name = save_image(crop_img, crop_image_path, crs)
            # save_png_image(file_name)
            new_name += 1
            pbar.update(1)

    return n + 1



In [9]:
from pathlib import Path

path = '../data/images/20230318_075512_29_249d_3B_AnalyticMS_SR_8b.tif'
output = '../outputs/tiled-satellite-images'


In [10]:
tile_image(path, output)

.tif
Input Image File Shape (D, H, W):(8, 8909, 12371)
crop_size = 256, stride = 256
Selected Image File Shape (D, H, W): 8, 8704, 12288
34
48


Generating:   1%|          | 11/1632 [00:00<00:15, 106.74img/s]

Overwrite existing file: ..\outputs\tiled-satellite-images\0001.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0002.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0003.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0004.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0005.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0006.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0007.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0008.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0009.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0010.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0011.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0012.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0013.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0014.tif
Overwrite existing file: ..\outputs\tiled-satell

Generating:   3%|▎         | 56/1632 [00:00<00:11, 141.60img/s]

Overwrite existing file: ..\outputs\tiled-satellite-images\0028.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0029.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0030.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0031.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0032.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0033.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0034.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0035.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0036.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0037.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0038.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0039.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0040.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0041.tif
Overwrite existing file: ..\outputs\tiled-satell

...

Overwrite existing file: ..\outputs\tiled-satellite-images\0057.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0058.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0059.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0060.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0061.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0062.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0063.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0064.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0065.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0066.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0067.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0068.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0069.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0070.tif
Overwrite existing file: ..\outputs\tiled-satell

Generating:   6%|▌         | 101/1632 [00:00<00:10, 144.80img/s]

Overwrite existing file: ..\outputs\tiled-satellite-images\0086.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0087.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0088.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0089.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0090.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0091.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0092.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0093.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0094.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0095.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0096.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0097.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0098.tif
Overwrite existing file: ..\outputs\tiled-satellite-images\0099.tif
Overwrite existing file: ..\outputs\tiled-satell

Generating: 100%|██████████| 1632/1632 [00:07<00:00, 209.15img/s]


1632